<a href="https://colab.research.google.com/github/harenlin/BBC-News-Classification-Using-BERT-Fine-Tuning/blob/main/BBC_News_BERT_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required package
!pip install bert-for-tf2

     |████████████████████████████████| 51kB 573kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=dae8b6d6d61344a71815e4ceecf6382d27cf9f77c0db038793419ef2aad4848c
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=c594a09315841dbd51e7c21f5d620bc97514400d94a3c184ac66ca44c1ac48fd
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=60f5d61c35000c4f48dbbc9edb8b4777915d4925418a131b6c28ab6677489d93
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
# Import modules
import os
import bert
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt

print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
pd.set_option('display.max_colwidth',1000)

TensorFlow Version: 2.5.0
Hub version:  0.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read the csv data into Pandas dataframe
df = pd.read_csv('/content/drive/My Drive/Tensorflow Course/BBC-News-Classification/BBC-News.csv')

In [ ]:
print(df.columns)

Index(['ArticleId', 'Text', 'Category'], dtype='object')


In [ ]:
print("The number of rows and columns in the training set is: {}".format(df.shape))

The number of rows and columns in the training set is: (1490, 3)


In [ ]:
# Identify missing values
df.apply(lambda x: sum(x.isnull()), axis=0)

ArticleId    0
Text         0
Category     0
dtype: int64

In [ ]:
df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness. cynthia cooper worldcom s ex-head of internal accounting alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy. prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper who now runs her own consulting business told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a green light to the procedures and practices used by worldcom. mr ...,business
1,154,german business confidence slides german business confidence fell in february knocking hopes of a speedy recovery in europe s largest economy. munich-based research institute ifo said that its confidence index fell to 95.5 in february from 97.5 in january its first decline in three months. the study found that the outlook in both the manufacturing and retail sectors had worsened. observers had been hoping that a more confident business sector would signal that economic activity was picking up. we re surprised that the ifo index has taken such a knock said dz bank economist bernd weidensteiner. the main reason is probably that the domestic economy is still weak particularly in the retail trade. economy and labour minister wolfgang clement called the dip in february s ifo confidence figure a very mild decline . he said that despite the retreat the index remained at a relatively high level and that he expected a modest economic upswing to continue. germany s economy grew...,business
2,1101,bbc poll indicates economic gloom citizens in a majority of nations surveyed in a bbc world service poll believe the world economy is worsening. most respondents also said their national economy was getting worse. but when asked about their own family s financial outlook a majority in 14 countries said they were positive about the future. almost 23 000 people in 22 countries were questioned for the poll which was mostly conducted before the asian tsunami disaster. the poll found that a majority or plurality of people in 13 countries believed the economy was going downhill compared with respondents in nine countries who believed it was improving. those surveyed in three countries were split. in percentage terms an average of 44% of respondents in each country said the world economy was getting worse compared to 34% who said it was improving. similarly 48% were pessimistic about their national economy while 41% were optimistic. and 47% saw their family s economic conditions i...,business
3,1976,lifestyle governs mobile choice faster better or funkier hardware alone is not going to help phone firms sell more handsets research suggests. instead phone firms keen to get more out of their customers should not just be pushing the technology for its own sake. consumers are far more interested in how handsets fit in with their lifestyle than they are in screen size onboard memory or the chip inside shows an in-depth study by handset maker ericsson. historically in the industry there has been too much focus on using technology said dr michael bjorn senior advisor on mobile media at ericsson s consumer and enterprise lab. we have to stop saying that these technologies will change their lives he said. we should try to speak to consumers in their own language and help them see how it fits in with what they are doing he told the bbc news website. for the study ericsson interviewed 14 000 mobile phone owners on the ways they use their phone. 

In [ ]:
df['Category'].unique()

array(['business', 'tech', 'politics', 'sport', 'entertainment'],
      dtype=object)

# BERT Input Data Requirement

In [ ]:
def get_ids(tokens, tokenizer, MAX_SEQ_LEN):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (MAX_SEQ_LEN - len(token_ids))
    return input_ids
  
def get_masks(tokens, MAX_SEQ_LEN):
    """Masks: 1 for real tokens and 0 for paddings"""
    return [1] * len(tokens) + [0] * (MAX_SEQ_LEN - len(tokens))

def get_segments(tokens, MAX_SEQ_LEN):
    """Segments: 0 for the first sequence, 1 for the second"""  
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]": current_segment_id = 1
    return segments + [0] * (MAX_SEQ_LEN - len(tokens))
  
def create_single_input(sentence, tokenizer, max_len):
    """Create an input from a sentence"""
    stokens = tokenizer.tokenize(sentence)
    stokens = stokens[:max_len] # max_len = MAX_SEQ_LEN - 2, why -2 ? ans: reserved for [CLS] & [SEP]
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
    return get_ids(stokens, tokenizer, max_len+2), get_masks(stokens, max_len+2), get_segments(stokens, max_len+2)

def convert_sentences_to_features(sentences, tokenizer, MAX_SEQ_LEN):
    """Convert sentences to features: input_ids, input_masks and input_segments"""
    input_ids, input_masks, input_segments = [], [], []
    for sentence in tqdm(sentences, position=0, leave=True):
      ids, masks, segments = create_single_input(sentence, tokenizer, MAX_SEQ_LEN-2) # why -2 ? ans: reserved for [CLS] & [SEP]
      input_ids.append(ids)
      input_masks.append(masks)
      input_segments.append(segments)
    return [np.asarray(input_ids, dtype=np.int32), np.asarray(input_masks, dtype=np.int32), np.asarray(input_segments, dtype=np.int32)]

In [ ]:
# hyper-parameters
MAX_SEQ_LEN = 512
NUM_OF_CLASS = len(df['Category'].unique())

# tokenizer
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file = os.path.join(gs_folder_bert, "vocab.txt"), do_lower_case=True)
# print("Vocab size:", len(tokenizer.vocab))
# tokens = tokenizer.tokenize("Hello TensorFlow!")
# print(tokens)
# ids = tokenizer.convert_tokens_to_ids(tokens)
# print(ids)
# print(tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]']))

In [ ]:
# create training and testing pairs
train_test_split = int((len(df)/10)*8)
x_train = convert_sentences_to_features(df['Text'][:train_test_split], tokenizer, MAX_SEQ_LEN)
x_test = convert_sentences_to_features(df['Text'][train_test_split:], tokenizer, MAX_SEQ_LEN)

def one_hot_encode(labels):
    encoded = []
    for label in labels:
        if label == 'tech': encoded.append([1,0,0,0,0])
        elif label == 'business': encoded.append([0,1,0,0,0])
        elif label == 'sport': encoded.append([0,0,1,0,0])
        elif label == 'entertainment': encoded.append([0,0,0,1,0])
        elif label == 'politics': encoded.append([0,0,0,0,1])
        else: print("Error!")
    return encoded

one_hot_encoded = one_hot_encode(df['Category'])
y_train, y_test = one_hot_encoded[:train_test_split], one_hot_encoded[train_test_split:]
y_train, y_test = np.array(y_train), np.array(y_test)
# check type
print(type(y_train), type(y_train[0]))

100%|██████████| 298/298 [00:02<00:00, 124.96it/s]

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


# Define Model Architecture

In [ ]:
def nlp_model(bert_base, num_of_class, MAX_SEQ_LEN):
    # Load the pre-trained BERT base model
    bert_layer = hub.KerasLayer(handle=bert_base, trainable=True)  
    # BERT layer three inputs: ids, masks and segments
    input_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_ids")           
    input_masks = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_masks")       
    input_segments = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="segment_ids")
    
    inputs = [input_ids, input_masks, input_segments] # BERT inputs
    pooled_output, sequence_output = bert_layer(inputs) # BERT outputs
    outputs = Dense(num_of_class, activation="softmax")(pooled_output) # output layer
    model = Model(inputs=inputs, outputs=outputs)
    return model

# model construction
bert_base = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
model = nlp_model(bert_base, 5, 512) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [ ]:
# use adam optimizer to minimize the categorical_crossentropy loss
optimizer = Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# fit the data to the model
history = model.fit(x_train, y_train, epochs=3, batch_size=8)

# Analyze Model Performence

In [ ]:
# predict on test dataset
from sklearn.metrics import classification_report
y_pred = np.argmax(model.predict(x_test), axis=1)

print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=['tech', 'business', 'sport', 'entertainment', 'politics']))